# Feature Selection
### Test different algorithmic functions to see which set of features will yeild better accuracy without overfitting the model. It will also reduce training time because of less data to process.
#### Credit to Jason Brownlee's [article](https://machinelearningmastery.com/feature-selection-machine-learning-python/) for guidance

In [34]:
import pandas as pd 
import numpy as np 

In [17]:
# Read in the csv file
df = pd.read_csv('../data/daily_summary.csv')
df = df.dropna(axis='columns', how='all')
df = df.drop(['index', 'date', 's&p_%change'], axis=1)
print(df.columns)
df.head()

Index(['tweets', 'isRetweet', 'isDeleted', 'avg_sentiment_score',
       'sum_sentiment_score', 'delta_avg', 'retweets', 'favorites',
       'word_count', 'positive', 'negative', 'neutral', 's&p_up/down'],
      dtype='object')


,tweets,isRetweet,isDeleted,avg_sentiment_score,sum_sentiment_score,delta_avg,retweets,favorites,word_count,positive,negative,neutral,s&p_up/down
0,17,4,0,5.855472,915.44,0.099388,522435,1118266,908,12,0,4,up
1,3,1,0,5.599524,156.98,-0.156560,227982,700580,163,2,0,1,up
2,1,0,0,6.650000,39.90,0.893916,36535,142625,32,1,0,0,down
3,5,0,0,5.826134,355.76,0.070050,193625,832596,351,4,0,1,up
4,2,0,0,5.873654,146.84,0.117570,69857,254719,144,2,0,0,up


# Define Features and Labels

In [35]:
# Create an array from the values in the dataframe values
array = df.values
# Pull out features and labels from the columns with slice 
X= array[:, :-1]
y= array[:, -1]

In [36]:
# tweets_df = pd.read_csv('../data/tweet_df_cleaned_w_sentiment.csv')

In [37]:
# tweets_df.head()

# Extra Tree Classifier
Feature Importance with Bagged decision trees like Random Forest and Extra Trees can be used to esmiate the importance of features. 

Learn more about the [ExtraTreesClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html) class in the scikit-learn API.

In [21]:
from sklearn.ensemble import ExtraTreesClassifier
# Feature extraction, bind class to an object
# n_estimators are the number of trees in the forest
model1 = ExtraTreesClassifier(n_estimators=10)
# Fit the arrays with the model
model1.fit(X,y)
print('The following is a scaled score of the features at n_estimators=10')
print(model1.feature_importances_)

The following is a scaled score of the features at n_estimators=10
[0.0843478  0.06937288 0.04722976 0.10500569 0.08420277 0.08778964
 0.09851328 0.09012433 0.10473973 0.08694547 0.0600661  0.08166256]


In [22]:
# Feature extraction, bind class to an object
# n_estimators are the number of trees in the forest
model2 = ExtraTreesClassifier(n_estimators=100)
# Fit the arrays with the model
model2.fit(X,y)
print('The following is a scaled score of the features at n_estimators=100')
print(model2.feature_importances_)

The following is a scaled score of the features at n_estimators=100
[0.08328879 0.06851396 0.04960431 0.0956694  0.09280637 0.09481782
 0.09852353 0.09762052 0.09489748 0.08169316 0.06113143 0.08143323]


In [23]:
df.columns

Index(['tweets', 'isRetweet', 'isDeleted', 'avg_sentiment_score',
       'sum_sentiment_score', 'delta_avg', 'retweets', 'favorites',
       'word_count', 'positive', 'negative', 'neutral', 's&p_up/down'],
      dtype='object')

In [31]:
# Rank the outcomes by n_estimators 100
feature_num = np.arange(1, 13, 1)
dict = {
    'feature_num' : feature_num,
    'feature name' : df.columns[0:-1],
    'n=10' : model1.feature_importances_, 
    'n=100' : model2.feature_importances_}
etc_rank = pd.DataFrame(dict)
etc_rank.sort_values(by='n=100', ascending=False)

,feature_num,feature name,n=10,n=100
6,7,retweets,0.098513,0.098524
7,8,favorites,0.090124,0.097621
3,4,avg_sentiment_score,0.105006,0.095669
8,9,word_count,0.104740,0.094897
5,6,delta_avg,0.087790,0.094818
4,5,sum_sentiment_score,0.084203,0.092806
0,1,tweets,0.084348,0.083289
9,10,positive,0.086945,0.081693
11,12,neutral,0.081663,0.081433
1,2,isRetweet,0.069373,0.068514


In [30]:
# Rank the outcomes by n_estimators 100
feature_num = np.arange(1, 13, 1)
dict = {
    'feature_num' : feature_num,
    'feature name' : df.columns[0:-1],
    'n=10' : model1.feature_importances_, 
    'n=100' : model2.feature_importances_}
etc_rank = pd.DataFrame(dict)
etc_rank.sort_values(by='n=10', ascending=False)

,feature_num,feature name,n=10,n=100
3,4,avg_sentiment_score,0.105006,0.095669
8,9,word_count,0.104740,0.094897
6,7,retweets,0.098513,0.098524
7,8,favorites,0.090124,0.097621
5,6,delta_avg,0.087790,0.094818
9,10,positive,0.086945,0.081693
0,1,tweets,0.084348,0.083289
4,5,sum_sentiment_score,0.084203,0.092806
11,12,neutral,0.081663,0.081433
1,2,isRetweet,0.069373,0.068514


## Univariate Selection
Statistical tests can be used to select those features that have the strongest relationship with the output variable.

The scikit-learn library provides the [SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class that can be used with a suite of different statistical tests to select a specific number of features.

Many different statistical test scan be used with this selection method. For example the ANOVA F-value method is appropriate for numerical inputs and categorical data, as we see in the Pima dataset. This can be used via the [f_classif()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html) function. We will select the 4 best features using this method in the example below.

In [38]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
test = SelectKBest(score_func=f_classif, k=5)
fit = test.fit(X, y)
# summarize scores
set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:5,:])

[0.713 0.374 0.95  0.166 0.208 0.166 3.283 4.348 0.224 0.011 0.165 1.718]
[[17 0 522435 1118266 4]
 [3 0 227982 700580 1]
 [1 0 36535 142625 0]
 [5 0 193625 832596 1]
 [2 0 69857 254719 0]]


In [42]:
df.columns[:-1]

Index(['tweets', 'isRetweet', 'isDeleted', 'avg_sentiment_score',
       'sum_sentiment_score', 'delta_avg', 'retweets', 'favorites',
       'word_count', 'positive', 'negative', 'neutral'],
      dtype='object')

In [43]:
dict = {
    'feature_num' : feature_num,
    'feature_name' : df.columns[:-1],
    'fit_scores' : fit.scores_
    }
etc_rank = pd.DataFrame(dict)
etc_rank #.sort_values(by='fit_scores', ascending=False)

,feature_num,feature_name,fit_scores
0,1,tweets,0.713286
1,2,isRetweet,0.373834
2,3,isDeleted,0.949567
3,4,avg_sentiment_score,0.165626
4,5,sum_sentiment_score,0.207855
5,6,delta_avg,0.165626
6,7,retweets,3.282661
7,8,favorites,4.347726
8,9,word_count,0.223576
9,10,positive,0.011185


## Recursive Feature Elimination
The Recursive Feature Elimination (or RFE) works by recursively removing attributes and building a model on those attributes that remain.

It uses the model accuracy to identify which attributes (and combination of attributes) contribute the most to predicting the target attribute.

You can learn more about the [RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE) class in the scikit-learn documentation.

The example below uses RFE with the logistic regression algorithm to select the top 3 features. The choice of algorithm does not matter too much as long as it is skillful and consistent.

In [44]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# feature extraction
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 6)
fit = rfe.fit(X, y)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Num Features: 6
Selected Features: [ True False False False  True False  True  True  True  True False False]
Feature Ranking: [1 2 3 5 1 7 1 1 1 1 4 6]


C:\Users\jaysu\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass n_features_to_select=6 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [46]:
dict = {
    'feature_num' : feature_num,
    'feature_name' : df.columns[:-1],
    'best_fit' : fit.support_
    }
feature_rfe = pd.DataFrame(dict)
is_true = feature_rfe['best_fit'] == True
feature_rfe[is_true]

,feature_num,feature_name,best_fit
0,1,tweets,True
4,5,sum_sentiment_score,True
6,7,retweets,True
7,8,favorites,True
8,9,word_count,True
9,10,positive,True


## Principal Component Analysis

[Principal Component Analysis](https://machinelearningmastery.com/calculate-principal-component-analysis-scratch-python/) (or PCA) uses linear algebra to transform the dataset into a compressed form.

Generally this is called a data reduction technique. A property of PCA is that you can choose the number of dimensions or principal component in the transformed result.

In the example below, we use PCA and select 3 principal components.

Learn more about the [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class in scikit-learn by reviewing the PCA API. Dive deeper into the math behind PCA on the [Principal Component Analysis Wikipedia](https://en.wikipedia.org/wiki/Principal_component_analysis) article.

In [47]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
fit = pca.fit(X)
# summarize components
print("Explained Variance: %s" % fit.explained_variance_ratio_)
print(fit.components_)

Explained Variance: [9.784e-01 2.164e-02 1.050e-06]
[[ 1.616e-05  7.440e-06  5.190e-07 -7.992e-08  9.483e-04 -7.992e-08
   3.142e-01  9.494e-01  1.011e-03  6.800e-06  2.214e-06  5.778e-06]
 [-7.977e-05 -7.759e-05 -3.555e-06  2.925e-07 -3.254e-03  2.925e-07
  -9.494e-01  3.142e-01 -3.533e-03 -3.039e-05 -1.673e-05 -3.201e-05]
 [ 7.865e-03  4.591e-03  5.223e-04 -1.718e-05  6.907e-01 -1.718e-05
  -4.996e-03  1.934e-04  7.231e-01  4.801e-03  5.484e-04  3.236e-03]]


In [46]:
from sklearn.ensemble import ExtraTreesClassifier
# Create an array from the values in the dataframe values
array = df.values
# Pull out features and labels from the columns with slice 
X= array[:, 1:]
y= array[:, 0]